In [27]:
# # task6
# import cv2
# bg_subtractor = cv2.createBackgroundSubtractorMOG2()
# # bg_subtractor = cv2.createBackgroundSubtractorKNN()
# cap = cv2.VideoCapture(0)
# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
    
#     fg_mask = bg_subtractor.apply(frame)
    

#     cv2.imshow('Foreground Mask', fg_mask)
    
#     if cv2.waitKey(30) & 0xFF == 27:
#         break

# cap.release()
# cv2.destroyAllWindows()

In [1]:
import cv2 
import numpy as np 
import imutils
import winsound 
import threading 
import time 

In [10]:
mode = [0,1] # 0 is web 1 is front 
cap = cv2.VideoCapture(mode[0], cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)

# get first frame 
# compare difference with the next frame 
# if difference is high enough = motion = alarm 
_, start_frame = cap.read() 
start_frame = imutils.resize(start_frame, width=500)
start_frame = cv2.cvtColor(start_frame, cv2.COLOR_BGR2GRAY) 
start_frame  = cv2.GaussianBlur(start_frame, (21,21),0)

alarm = False 
alarm_mode = False 
alarm_cnt = 0 

def beep(): 
    global alarm 
    for _ in range(5): 
        if not alarm_mode: 
            break 
        winsound.Beep(2500,1000)
    alarm = False 

while True:
    _, frame = cap.read() 
    frame = imutils.resize(frame, width=500)
    if alarm_mode: 
        frame_bw = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_bw = cv2.GaussianBlur(frame_bw, (5,5),0)
        difference = cv2.absdiff(frame_bw, start_frame)
        threshold = cv2.threshold(difference, 25,255,cv2.THRESH_BINARY)[1]
        start_frame = frame_bw 

        if threshold.sum() > 300: # smaller the number the sensitive the detection 
            alarm_cnt +=1 
        else: 
            if alarm_cnt > 0: 
                alarm_cnt-=1 

        cv2.imshow('Cam', threshold)
    else: 
        cv2.imshow('cam', frame)
    
    if alarm_cnt > 20: 
        if not alarm: 
            alarm = True 
            threading.Thread(target=beep).start() 
    
    key = cv2.waitKey(30)
    if key == ord('t'): 
        time.sleep(5)
        alarm_mode = not alarm_mode
        alarm_cnt = 0 
    
    if key == ord('q'): 
        alarm_mode = False 
        break 

cap.release()
cv2.destroyAllWindows()


